<a href="https://colab.research.google.com/github/manoharpavuluri/AI_NLP_SQL_LLM/blob/main/AI_SQL_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Libraries**

In [1]:
!pip install langchain_openai langchain_community langchain pyodbc pymysql chromadb python-dotenv -q
!pip install --upgrade --quiet langchain-google-cloud-sql-mssql langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 856.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.8 MB/s eta 0:0

In [2]:
from google.colab import drive
from google.colab import userdata

import os

drive.mount('/content/drive')

# Then read the .env file securely from your Google Drive
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/Colab Notebooks/AI_SQL_LLM/config.env')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [3]:

# Access the credentials
mysql_db_user = os.getenv("MYSQL_DB_USER")
mysql_db_pass = os.getenv("MYSQL_DB_PASS")
mysql_db_host = os.getenv("MYSQL_DB_HOST")
mysql_db_name = os.getenv("MYSQL_DB_NAME")

# **Making SQL Connection using Langchain**



In [4]:
from langchain_community.utilities.sql_database import SQLDatabase

mysql_db = SQLDatabase.from_uri(f"mysql+pymysql://{mysql_db_user}:{mysql_db_pass}@{mysql_db_host}/{mysql_db_name}")


In [5]:
print(mysql_db.dialect)


mysql


In [6]:
print(mysql_db.get_usable_table_names())


['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']


In [20]:
# print(mysql_db.table_info)

In [19]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN')


In [10]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

# **Adding OpenAI Chat mode**
**to turn sentence into query**

In [17]:
# using the model from ChatOpenAI, and mysql_db from langchain_community,
# you are converting the 'question' to sql query

llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature =0)
generate_query = create_sql_query_chain(llm,mysql_db)
query = generate_query.invoke({"question": "what is price of '1968 Ford Mustang'"})
print(query)

SELECT `buyPrice`, `MSRP` 
FROM products 
WHERE `productName` = '1968 Ford Mustang'
LIMIT 1;


In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db= mysql_db)
execute_query.invoke(query)

"[(Decimal('95.34'), Decimal('194.57'))]"

In [13]:
chain = generate_query | execute_query
chain.invoke({"question": "How many orders are there?"})

'[(326,)]'

In [14]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

# **Adding prompt template**

In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
  """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

  Question: {question}
  SQL Query: {query}
  SQL Result: {result}
  Answer: """
)


In [16]:
rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query = generate_query).assign(
        result = itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "How many customers have an order count greater than 5"})

'There are 2 customers who have an order count greater than 5.'

# **Enhancing NL2SQL Models with Few-Shot Examples**

In [23]:
 examples = [
     {
         "input": "List all customers in France with a credit limit over 20,000.",
         "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
     },
     {
         "input": "Get the highest payment amount made by any customer.",
         "query": "SELECT MAX(amount) FROM payments;"
     }
 ]


In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate, PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    # input_variables = ["input", "top_k"],
    input_variables = ["input"],
)
print(few_shot_prompt.format(input1 = "how many products are there?"))

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;


In [28]:
 from langchain_community.vectorstores import Chroma
 from langchain_core.example_selectors import SemanticSimilarityExampleSelector
 from langchain_openai import OpenAIEmbeddings

 vectorstore = Chroma()
 vectorstore.delete_collection()
 example_selector = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(),
     vectorstore,
     k=2,
     input_keys=["input"],
 )
 example_selector.select_examples({"input": "how many employees we have?"})
 few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     example_selector=example_selector,
     input_variables=["input","top_k"],
 )
 print(few_shot_prompt.format(input="How many products are there?"))


Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;


In [30]:
 final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
         few_shot_prompt,
         ("human", "{input}"),
     ]
 )
 print(final_prompt.format(input="How many products are there?",table_info="some table info"))
 generate_query = create_sql_query_chain(llm, mysql_db,final_prompt)
 chain = (
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )
 chain.invoke({"question": "How many csutomers with credit limit more than 50000"})


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: How many products are there?


'There are 85 customers with a credit limit greater than 50000.'

In [33]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI_SQL_LLM/database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)


Table Name:employees
Table Description:employees




In [34]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer and their order count"})
tables


<ipython-input-34-3d47447c16a4>:8: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)


[Table(name='customers'), Table(name='orders')]

In [35]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer and their order count"})


['customers', 'orders']

In [36]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cutomers with order count more than 5"})


'There are 2 customers with an order count of more than 5.'

In [37]:
 from langchain.memory import ChatMessageHistory
 history = ChatMessageHistory()


In [38]:
 final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
         few_shot_prompt,
         MessagesPlaceholder(variable_name="messages"),
         ("human", "{input}"),
     ]
 )
 print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: How many products are there?


In [40]:
 generate_query = create_sql_query_chain(llm, mysql_db,final_prompt)

 chain = (
 RunnablePassthrough.assign(table_names_to_use=select_table) |
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )


In [42]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})


In [43]:
history.add_user_message(question)
history.add_ai_message(response)
history.messages

[HumanMessage(content='How many cutomers with order count more than 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='There are 2 customers with an order count of more than 5.', additional_kwargs={}, response_metadata={})]

In [44]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response

"The names of the customers who have placed more than 5 orders are 'Mini Gifts Distributors Ltd.' and 'Euro+ Shopping Channel'."